# Knapsack Julia

In [141]:
using JuMP, GLPK, Cbc, Test

In [142]:
const MOI = JuMP.MathOptInterface

MathOptInterface

In [143]:
function example_cannery(; verbose = true)
    plants = ["Seattle", "San-Diego"]
    markets = ["New-York", "Chicago", "Topeka"]

    # Capacity and demand in cases.
    capacity = [350, 600]
    demand = [300, 300, 300]

    # Distance in thousand miles.
    distance = [2.5 1.7 1.8; 2.5 1.8 1.4]

    # Cost per case per thousand miles.
    freight = 90

    num_plants = length(plants)
    num_markets = length(markets)

    cannery = Model(with_optimizer(Cbc.Optimizer))

    @variable(cannery, ship[1:num_plants, 1:num_markets] >= 0)

    # Ship no more than plant capacity
    @constraint(cannery, capacity_con[i in 1:num_plants],
        sum(ship[i,j] for j in 1:num_markets) <= capacity[i]
    )

    # Ship at least market demand
    @constraint(cannery, demand_con[j in 1:num_markets],
        sum(ship[i,j] for i in 1:num_plants) >= demand[j]
    )

    # Minimize transporatation cost
    @objective(cannery, Min, sum(distance[i, j] * freight * ship[i, j]
        for i in 1:num_plants, j in 1:num_markets)
    )

    JuMP.optimize!(cannery)

    if verbose
        println("RESULTS:")
        for i in 1:num_plants
            for j in 1:num_markets
                println("  $(plants[i]) $(markets[j]) = $(JuMP.value(ship[i, j]))")
            end
        end
    end

    @test JuMP.termination_status(cannery) == MOI.OPTIMAL
    @test JuMP.primal_status(cannery) == MOI.FEASIBLE_POINT
    @test JuMP.objective_value(cannery) == 151200.0
end

example_cannery (generic function with 1 method)

In [144]:
example_cannery(verbose=true)

RESULTS:
  Seattle New-York = 50.0
  Seattle Chicago = 300.0
  Seattle Topeka = 0.0
  San-Diego New-York = 250.0
  San-Diego Chicago = 0.0
  San-Diego Topeka = 300.0
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 5 (0) rows, 6 (0) columns and 12 (0) elements
0  Obj 0 Primal inf 900 (3)
4  Obj 151200
Optimal - objective value 151200
Optimal objective 151200 - 4 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



Test Passed

## Knapsack

In [3]:
struct item
    profit
    weight
end

In [86]:
function knapsack(items, capacity, opt=GLPK.Optimizer, verbose=false)
    profit = [x.profit for x in items]
    weight = [x.weight for x in items]
    model = Model(with_optimizer(opt))
    @variable(model, x[1:length(items)], Bin)
    @objective(model, Max, profit' * x)
    @constraint(model, weight' *x <= capacity)
    JuMP.optimize!(model)
    if verbose
        println("Objective is: ", JuMP.objective_value(model))
        println("Solution is:")
        for i in 1:length(items)
            print("x[$i] = ", JuMP.value(x[i]))
            println(", p[$i]/w[$i] = ", profit[i] / weight[i])
        end
    end
    return JuMP.objective_value(model)
end

knapsack (generic function with 3 methods)

In [134]:
using Profile
using ProfileView

In [135]:
num_items = 70000;
items = [item(rand(1:20), rand(1:20)) for i in 1:num_items];
cap = rand(1:20*num_items);

In [139]:
knapsack(items, cap, Cbc.Optimizer)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 547218 - 1.21 seconds
Cgl0004I processed model has 1 rows, 163438 columns (163438 integer (133600 of which binary)) and 163438 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.2
Cbc0038I Solution found of -547218
Cbc0038I Branch and bound needed to clear up 1 general integers
Cbc0038I Full problem 1 rows 163438 columns, reduced to 1 rows 24649 columns
Cbc0038I Cleaned solution of -547217
Cbc0038I Before mini branch and bound, 163437 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (46.77 seconds)
Cbc0038I After 46.78 seconds - Feasibility pump exiting with objective of -547217 - took 1.82 seconds
Cbc0012I Integer solution of -547217 found by feasibility pump after 0 iterations and 0 nodes (46.79 seconds)
Cbc00

547217.0